# OLMo Model Dataset Parsing

This notebook loads and processes the OLMo dataset from Hugging Face for use with the ICAI framework.

In [1]:
# Install any missing packages
# !pip install datasets transformers pandas numpy matplotlib

In [34]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from tqdm.auto import tqdm

## Load the OLMo Dataset from Hugging Face

The Allen Institute for AI (AI2) released the OLMo model along with several datasets. Let's load the main dataset from Hugging Face.

In [ ]:
# Load the OLMo dataset from Hugging Face
# You can choose a specific config/subset if needed
olmo_dataset = load_dataset("allenai/olmo-2-0325-32b-preference-mix")

# Print basic dataset info
print(f"Dataset structure: {olmo_dataset}")

In [36]:
df = olmo_dataset["train"].to_pandas()

In [37]:
df_10k = df.sample(10000)

In [ ]:
# rename columns from chosen to rejected to "text_a" and "text_b", and add columns "preferred_text" which is always "text_a"
df_10k = df_10k.rename(columns={"chosen": "text_a", "rejected": "text_b"})
df_10k["preferred_text"] = "text_a"

# add original index as "id" column
df_10k["id"] = df_10k.index

# randomly switch text_a and text_b and adapt corresponding preferred_text label
# Create a random mask to determine which rows to switch
np.random.seed(691)
switch_mask = np.random.rand(len(df_10k)) < 0.5

# Create a copy of the dataframe to avoid modifying during iteration
df_10k_rand = df_10k.copy()

# Create a new dataframe with swapped values where switch_mask is True
df_10k_rand.loc[switch_mask, 'text_a'] = df_10k.loc[switch_mask, 'text_b']
df_10k_rand.loc[switch_mask, 'text_b'] = df_10k.loc[switch_mask, 'text_a']

# Update preferred_text for switched rows to "text_b"
df_10k_rand.loc[switch_mask, 'preferred_text'] = 'text_b'


# Print the number of rows that were switched
print(f"Switched text_a and text_b in {switch_mask.sum()} out of {len(df_10k_rand)} rows")


In [ ]:
# Validate that all chosen texts are actually chosen in the original dataset
validation_errors = 0

for idx, row in df_10k.iterrows():
    original_chosen = df.loc[row['id'], 'chosen']
    original_rejected = df.loc[row['id'], 'rejected']

    # Check if the preferred text matches the original chosen text
    if row['preferred_text'] == 'text_a':
        # Use string equality check with .iloc[0] to handle array-like values
        if not str(row['text_a']) == str(original_chosen):
            validation_errors += 1
    else:  # preferred_text is 'text_b'
        # Use string equality check with .iloc[0] to handle array-like values
        if not str(row['text_b']) == str(original_chosen):
            validation_errors += 1

print(f"Validation check: Found {validation_errors} errors out of {len(df_10k)} rows")
if validation_errors == 0:
    print("✓ All preferred texts correctly match the original chosen texts")
else:
    print("✗ Some preferred texts do not match the original chosen texts")


In [ ]:
df_10k_rand.to_csv("../data/processed/allenai/olmo2-0325-32b-preference-mix_10k_rand.csv", index=False)